<a href="https://colab.research.google.com/github/Donovanphenry/SCAI-Net/blob/main/rainforest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### We can construct a mosaic of nearby tiles using this method: https://www.kaggle.com/c/planet-understanding-the-amazon-from-space/discussion/36738

# Preprocess data
---

# Construct model
---

# Train model
---

# View results
---